In [2]:
from pymatgen.io.lammps.data import *

In [9]:
system = LammpsData.from_file("data.oleic_xligand.test")
xlig_dat = LammpsData.from_file("data.xligand")
oleic_dat = LammpsData.from_file("data.oleic_acid")

xlig = xlig_dat.structure
oleic = oleic_dat.structure

box, ff, tops = system.disassemble()
forcefield = ff.as_dict()

In [4]:
def get_ff_type(idx,structure):
    site = structure.as_dict()['sites']['sites'][idx] # preserves original index of atoms from lammps data file
    ff_type = site['properties']['ff_map']  # pull the forcefield type from the structure-topo dict
    return ff_type

def get_bond_type(bond,ff_dict):
    
    bond_type = ""
    
    for i in range(len(ff_dict['topo_coeffs']["Bond Coeffs"])):
        # print(bond, ff_dict['topo_coeffs']["Bond Coeffs"][i]['types'][0]) 
        if bond == ff_dict['topo_coeffs']["Bond Coeffs"][i]['types'][0]:
            bond_type = i
        elif bond[::-1] == ff_dict['topo_coeffs']["Bond Coeffs"][i]['types'][0]:
            bond_type = i
    
    if type(bond_type) == int:
        return bond_type
    else:
        raise Exception("type not found")

In [17]:
# this function expects:
# -- relation should be an array of atom types as imported by pymatgen from lammps datafile 
# -- eg [C1, H1, C1, N1]
# -- ff_dict should be the output of the ff.as_dict() pymatgen lammps forcefield object 
# -- topology should be the name of the relation you want to type (Bonds, Angles, Dihedrals, Impropers)
def get_topo_type(relation, ff_dict, topology):
    
    topo_type = ""
    topo_coeff = ""
    
    # convert standard topology name to singular + " Coeff" format for pymatgen ff_dict format
    topo = {"Bond", "Angle", "Dihedral", "Improper"}
    for i in topo:
        if i in topology:
            topo_coeff = i + " Coeffs"
    
    if topo_coeff == "":
        raise Exception("Topology not found")
    
    # check the original or reverse order of the topological relation (bond, angle...) 
    for i in range(len(ff_dict['topo_coeffs'][topo_coeff])):
        # print(relation, ff_dict['topo_coeffs'][topo_coeff][i]['types'][0])
        if relation in ff_dict['topo_coeffs'][topo_coeff][i]['types']:
            topo_type = i
        elif relation[::-1] in ff_dict['topo_coeffs'][topo_coeff][i]['types']:
            topo_type = i
    
    if type(topo_type) == int:
        return topo_type
    else:
        raise Exception("type not found")

In [16]:
dihed_typed = [get_ff_type(x,tops[0]) for x in tops[0].topologies["Dihedrals"][0]]
dihed_typed
get_topo_type(dihed_typed, forcefield, "Dihedrals")

3

In [22]:
# writes out the desired lammps topology in molecule-file compatible format 
# note the "+ 1"s to convert to lammps 1-based indexing
def get_lammps_topology(structure, ff_dict, topology):
    i = 0
    dat = []
    for neighbor in structure.topologies[topology]:
        i  += 1
        neighbor_types = [get_ff_type(x,structure) for x in neighbor] # get ff types for each element in the connected relation
        relation_type = get_topo_type(neighbor_types, ff_dict, topology) # get type of the relation
        lammps_bond = [str(i + 1) for i in neighbor] # convert to LAMMPS 1-based indexing
        # print(i, relation_type + 1, " ".join(lammps_bond))
        dat.append(f"{i} {relation_type + 1} {' '.join(lammps_bond)}\n")
    
    return dat

def write_lammps_topology(filename, topo_dat):
    with open(filename, "w") as f:
        for line in topo_dat:
            f.write(line)

In [23]:
dat = get_lammps_topology(tops[0], forcefield, "Bonds")
write_lammps_topology("xlig_bonds_test.dat", dat)
dat[0:5]

['1 1 1 2\n', '2 1 1 31\n', '3 2 1 32\n', '4 2 1 33\n', '5 1 2 3\n']

In [6]:
def generate_neighbors(topology,structure):
    neighbors = {atom: set() for atom in range(1,len(structure)+1)}
    for relation in topology:
        neighbors[relation[0] + 1].add(relation[-1] + 1)
        neighbors[relation[-1] + 1].add(relation[0] + 1)
    return neighbors

In [7]:
def get_special_bonds(neighbors,structure):
    special_bonds = {atom: set() for atom in range(1, len(structure) + 1)}
    for top in ['Bonds', 'Angles', 'Dihedrals', 'Impropers']:
        for atom in special_bonds:
            special_bonds[atom] = special_bonds[atom].union(neighbors[top][atom])
    
    return special_bonds


def get_special_bond_counts(neighbors, structure):
    special_bond_counts = {atom: [] for atom in range(1, len(structure) + 1)}
    for top in ['Bonds', 'Angles', 'Dihedrals']:
        for atom in special_bond_counts:
            if top == "Bonds" or top == "Angles":
                special_bond_counts[atom].append(len(neighbors[top][atom]))
            elif top == "Dihedrals":
                count = 0
                for i in neighbors[top][atom]:
                    if i not in neighbors["Angles"][atom]:
                        count += 1
                special_bond_counts[atom].append(count)
    
    return special_bond_counts

In [12]:
topologies = ['Bonds', 'Angles', 'Dihedrals', 'Impropers']
neighbors = {topology: generate_neighbors(tops[0].topologies[topology], xlig) for topology in topologies}
special_bonds = get_special_bonds(neighbors)
special_bond_counts = get_special_bond_counts(neighbors)

In [21]:
neighbors['Impropers']

{1: {20},
 2: {20},
 3: set(),
 4: set(),
 5: set(),
 6: set(),
 7: set(),
 8: set(),
 9: set(),
 10: set(),
 11: {15},
 12: set(),
 13: set(),
 14: {17},
 15: {11, 17, 45},
 16: set(),
 17: {14, 15, 48},
 18: {20},
 19: set(),
 20: {1, 2, 18},
 21: set(),
 22: set(),
 23: set(),
 24: set(),
 25: set(),
 26: set(),
 27: set(),
 28: set(),
 29: set(),
 30: set(),
 31: set(),
 32: set(),
 33: set(),
 34: set(),
 35: set(),
 36: set(),
 37: set(),
 38: set(),
 39: set(),
 40: set(),
 41: set(),
 42: set(),
 43: set(),
 44: set(),
 45: {15},
 46: set(),
 47: set(),
 48: {17},
 49: set(),
 50: set(),
 51: set(),
 52: set(),
 53: set(),
 54: set()}

In [13]:
coords = []
types = []
charges = []
with open("coords.txt") as f:
    dat = f.readlines()
    
for line in dat:
    raw = line.split()
    x = round(float(raw[3]),1)
    y = round(float(raw[4]),1)
    z = round(float(raw[5]),1)
    pos = [int(raw[0]), x,y,z]
    at = [int(raw[0]), int(raw[2])]
    charge = [int(raw[0]), float(raw[6])]
    charges.append(charge)
    types.append(at)
    coords.append(pos)

In [14]:
with open("clean_coords.txt", "w") as f:
    f.write("Coords\n\n")
    for pos in coords:
        f.write(f"{pos[0]}    {pos[1]} {pos[2]} {pos[3]}\n")
    f.write("\nTypes\n\n")
    for at in types:
        f.write(f"{at[0]}    {at[1]}\n")
    f.write("\nCharges\n\n")
    for charge in charges:
        f.write(f"{charge[0]}    {charge[1]}\n")
    f.write("\nSpecial Bond Counts\n\n")
    for atom in special_bond_counts:
        f.write(f"{atom}    {' '.join([str(x) for x in special_bond_counts[atom]])}\n")
    f.write("\nSpecial Bonds\n\n")
    for atom in special_bonds:
        f.write(f"{atom}    {' '.join([str(x) for x in special_bonds[atom]])}\n")

In [17]:
len(tops[1].topologies["Bonds"])

53

In [19]:
topologies = ['Bonds', 'Angles', 'Dihedrals', 'Impropers']
neighbors = {topology: generate_neighbors(tops[1].topologies[topology], oleic) for topology in topologies}
special_bonds = get_special_bonds(neighbors, oleic)
special_bond_counts = get_special_bond_counts(neighbors, oleic)

In [22]:
neighbors['Impropers']

{1: {20},
 2: {20},
 3: set(),
 4: set(),
 5: set(),
 6: set(),
 7: set(),
 8: set(),
 9: set(),
 10: set(),
 11: {15},
 12: set(),
 13: set(),
 14: {17},
 15: {11, 17, 45},
 16: set(),
 17: {14, 15, 48},
 18: {20},
 19: set(),
 20: {1, 2, 18},
 21: set(),
 22: set(),
 23: set(),
 24: set(),
 25: set(),
 26: set(),
 27: set(),
 28: set(),
 29: set(),
 30: set(),
 31: set(),
 32: set(),
 33: set(),
 34: set(),
 35: set(),
 36: set(),
 37: set(),
 38: set(),
 39: set(),
 40: set(),
 41: set(),
 42: set(),
 43: set(),
 44: set(),
 45: {15},
 46: set(),
 47: set(),
 48: {17},
 49: set(),
 50: set(),
 51: set(),
 52: set(),
 53: set(),
 54: set()}

In [20]:
coords = []
types = []
charges = []
with open("oleic_coords.txt") as f:
    dat = f.readlines()
    
for line in dat:
    raw = line.split()
    x = round(float(raw[3]),1)
    y = round(float(raw[4]),1)
    z = round(float(raw[5]),1)
    pos = [int(raw[0]), x,y,z]
    at = [int(raw[0]), int(raw[2])]
    charge = [int(raw[0]), float(raw[6])]
    charges.append(charge)
    types.append(at)
    coords.append(pos)

with open("oleic_dat.txt", "w") as f:
    f.write("Coords\n\n")
    for pos in coords:
        f.write(f"{pos[0]}    {pos[1]} {pos[2]} {pos[3]}\n")
    f.write("\nTypes\n\n")
    for at in types:
        f.write(f"{at[0]}    {at[1]}\n")
    f.write("\nCharges\n\n")
    for charge in charges:
        f.write(f"{charge[0]}    {charge[1]}\n")
    f.write("\nSpecial Bond Counts\n\n")
    for atom in special_bond_counts:
        f.write(f"{atom}    {' '.join([str(x) for x in special_bond_counts[atom]])}\n")
    f.write("\nSpecial Bonds\n\n")
    for atom in special_bonds:
        f.write(f"{atom}    {' '.join([str(x) for x in special_bonds[atom]])}\n")